In [32]:
import pandas as pd
import numpy as np

In [33]:
data_directory = 'data/'
stations = pd.read_csv(f'{data_directory}/station_with_demand.csv')
distance_matrix = pd.read_csv(f'{data_directory}/distance_matrix.csv', index_col=0).to_numpy()

In [34]:
stations

,station_id,station_name,capacity,lng,lat,MEAN_WEEKLY_TRIPS,NORMALISED_CAPACITY,DEMAND_SCORE,DEMAND_CATEGORY
0,5772.05,Morton St & Greenwich St,56,-74.008870,40.731150,0,0.441667,0.000000,Lowest
1,6560.15,Broadway & W 38 St,68,-73.987349,40.752973,511,0.541667,234.719333,Low
2,7727.07,Amsterdam Ave & W 119 St,29,-73.959621,40.808625,0,0.216667,0.000000,Lowest
3,6569.08,W 35 St & Dyer Ave,41,-73.997402,40.754692,206,0.316667,140.972667,Low
4,7023.04,W 59 St & 10 Ave,117,-73.988038,40.770513,341,0.950000,17.391000,Lowest
...,...,...,...,...,...,...,...,...,...
657,8416.10,W 186 St & St Nicholas Ave,26,-73.931308,40.852253,8,0.191667,6.474667,Lowest
658,4846.01,South St & Whitehall St,55,-74.012342,40.701221,313,0.433333,177.679667,Low
659,6890.01,W 53 St & 10 Ave,29,-73.990617,40.766697,424,0.216667,332.557333,Medium
660,6483.06,W 36 St & 7 Ave,38,-73.989539,40.752149,173,0.291667,122.714667,Low


In [35]:
print(distance_matrix)

[[    0.         13914.30811197 41865.43844055 ... 18008.20184989
  13007.34927655  9790.09398049]
 [13914.30811197     0.         27951.13032858 ...  5906.00201928
    906.95883542  4124.21413149]
 [41865.43844055 27951.13032858     0.         ... 23857.23659066
  28858.08916399 32075.34446006]
 ...
 [18008.20184989  5906.00201928 23857.23659066 ...     0.
   5599.49299432  8218.1078694 ]
 [13007.34927655   906.95883542 28858.08916399 ...  5599.49299432
      0.          3217.25529607]
 [ 9790.09398049  4124.21413149 32075.34446006 ...  8218.1078694
   3217.25529607     0.        ]]


In [36]:
# Assigning percentages based on demand category
percentage_map = {
    'Lowest': 0.10,
    'Low': 0.30,
    'Medium': 0.50,
    'Highest': 0.70
}

In [37]:
stations['DEMAND'] = stations['DEMAND_CATEGORY'].map(percentage_map) * stations['capacity']
# stations['DEMAND'] = round(stations['DEMAND'],0).astype(int)

In [38]:
stations

,station_id,station_name,capacity,lng,lat,MEAN_WEEKLY_TRIPS,NORMALISED_CAPACITY,DEMAND_SCORE,DEMAND_CATEGORY,DEMAND
0,5772.05,Morton St & Greenwich St,56,-74.008870,40.731150,0,0.441667,0.000000,Lowest,5.6
1,6560.15,Broadway & W 38 St,68,-73.987349,40.752973,511,0.541667,234.719333,Low,20.4
2,7727.07,Amsterdam Ave & W 119 St,29,-73.959621,40.808625,0,0.216667,0.000000,Lowest,2.9
3,6569.08,W 35 St & Dyer Ave,41,-73.997402,40.754692,206,0.316667,140.972667,Low,12.3
4,7023.04,W 59 St & 10 Ave,117,-73.988038,40.770513,341,0.950000,17.391000,Lowest,11.7
...,...,...,...,...,...,...,...,...,...,...
657,8416.10,W 186 St & St Nicholas Ave,26,-73.931308,40.852253,8,0.191667,6.474667,Lowest,2.6
658,4846.01,South St & Whitehall St,55,-74.012342,40.701221,313,0.433333,177.679667,Low,16.5
659,6890.01,W 53 St & 10 Ave,29,-73.990617,40.766697,424,0.216667,332.557333,Medium,14.5
660,6483.06,W 36 St & 7 Ave,38,-73.989539,40.752149,173,0.291667,122.714667,Low,11.4


In [39]:
import pulp

def calculate_expected_demand(N, base_demand, weather_factors, weather_probabilities):
    """
    Calculate the expected demand for each station for a single day using demand factors 
    and constant weather probabilities.
    """
    expected_demand = np.zeros(N)
    
    # Calculate expected demand based on weather factors and probabilities
    for i in range(N):
        station_demand = 0
        for weather, prob in weather_probabilities.items():
            # Demand in Weather  = Base Demand * Factor in Weather
            demand_in_weather = base_demand[i] * weather_factors[weather]
            # Weighted Demand in Weather = Probability of Weather * Demand in Weather
            weighted_demand = prob * demand_in_weather
            # Add to station's demand
            station_demand += weighted_demand
        # Total probability of different weather should add up to 1
        expected_demand[i] = station_demand
    
    return expected_demand

def optimize_bike_relocation(N, current_bikes, expected_demand, relocation_cost):
    """
    Optimize bike relocation between stations to meet the expected demand while 
    minimizing the relocation cost.
    """
    # Optimization problem
    problem = pulp.LpProblem("Total_Bike_Relocation_Cost", pulp.LpMinimize)

    # Decision variables: Number of bikes to move from station i to station j
    # X >= 0
    # X is an integer
    x = pulp.LpVariable.dicts("x", [(i, j) for i in range(N) for j in range(N)], lowBound=0, cat='Integer')

    # Objective Function: Minimize total relocation cost
    problem += pulp.lpSum(relocation_cost[i][j] * x[(i, j)] for i in range(N) for j in range(N))

    # Constraints:
    # 1. Supply constraints: Total bikes moved from station i should not exceed current inventory
    for i in range(N):
        problem += pulp.lpSum(x[(i, j)] for j in range(N)) <= current_bikes[i]

    # 2. Demand constraints: Total bikes moved to station j should meet the expected demand
    for j in range(N):
        problem += pulp.lpSum(x[(i, j)] for i in range(N)) >= expected_demand[j]
    
    # 3. Prevent self-relocation: No bikes should be moved from a station to itself
    for i in range(N):
        problem += x[(i, i)] == 0

    # Solve the optimization problem
    problem.solve()

    # Extract the solution
    relocation_plan = {(i, j): x[(i, j)].varValue for i in range(N) for j in range(N) if x[(i, j)].varValue > 0}
    
    return relocation_plan

def main():
    """
    Main function to demonstrate the full Stage 2 model for bike reallocation between stations.
    """
    # Step 1: Define the inputs for demand calculation
    N = len(stations)  # Number of stations
    base_demand = stations["DEMAND"]  # Current demand for each station
    print("Original Base Demand")
    for i, demand in enumerate(base_demand):
        print(f"Station {i + 1}: {demand:.2f} bikes")
    
    # Define demand factors for different weather conditions
    weather_factors = {
        "sunny": 1.2,   # 20% increase in demand on sunny days
        "rainy": 0.7,   # 30% decrease in demand on rainy days
        "cloudy": 1.0   # No change in demand on cloudy days
    }

    # Define constant weather probabilities
    weather_probabilities = {
        "sunny": 0.20,
        "rainy": 0.30,
        "cloudy": 0.50
    }

    # Step 2: Calculate the expected demand for a single day
    expected_demand = calculate_expected_demand(N, base_demand, weather_factors, weather_probabilities)
    
    print("Expected Demand for a Single Day:")
    for i, demand in enumerate(expected_demand):
        print(f"Station {i + 1}: {demand:.2f} bikes")
    
    # Step 3: Define inputs for the relocation model
    # Current number of bikes at each station <- taken from output of Stage 1
    # Expected Format: A list of bikes at N Stations: [2 , 7 , ... , 3] with n elements
    current_bikes = np.random.randint(10, 70, size=N)
    
    # Cost matrix for relocation
    # Expected Format: A N * N matrix of relocation cost from station i to station j
    # Example format:
    # [[1, 2, 3, 4, 5]
    # [2, 1, 2, 3, 4]
    # [3, 2, 1, 2, 3]
    # [4, 3, 2, 1, 2]
    # [5, 4, 3, 2, 1]]
    relocation_cost = distance_matrix
    print("\nRelocation Cost Matrix:")
    print(relocation_cost)

    # Step 4: Run the optimization model for bike relocation
    relocation_plan = optimize_bike_relocation(N, current_bikes, expected_demand, relocation_cost)
    
    print("\nOptimal Bike Relocation Plan:")
    for (i, j), bikes in relocation_plan.items():
        print(f"Move {bikes:.0f} bikes from Station {i + 1} to Station {j + 1}")

# Run the main function
if __name__ == "__main__":
    main()


Original Base Demand
Station 1: 5.60 bikes
Station 2: 20.40 bikes
Station 3: 2.90 bikes
Station 4: 12.30 bikes
Station 5: 11.70 bikes
Station 6: 12.30 bikes
Station 7: 31.50 bikes
Station 8: 24.50 bikes
Station 9: 4.40 bikes
Station 10: 7.50 bikes
Station 11: 20.50 bikes
Station 12: 2.30 bikes
Station 13: 22.50 bikes
Station 14: 15.50 bikes
Station 15: 14.10 bikes
Station 16: 9.30 bikes
Station 17: 37.50 bikes
Station 18: 1.90 bikes
Station 19: 2.10 bikes
Station 20: 14.40 bikes
Station 21: 23.70 bikes
Station 22: 16.50 bikes
Station 23: 16.50 bikes
Station 24: 18.30 bikes
Station 25: 3.10 bikes
Station 26: 2.50 bikes
Station 27: 2.10 bikes
Station 28: 3.50 bikes
Station 29: 51.80 bikes
Station 30: 9.30 bikes
Station 31: 6.90 bikes
Station 32: 2.10 bikes
Station 33: 1.90 bikes
Station 34: 25.00 bikes
Station 35: 4.70 bikes
Station 36: 15.00 bikes
Station 37: 28.50 bikes
Station 38: 18.00 bikes
Station 39: 2.90 bikes
Station 40: 9.90 bikes
Station 41: 21.90 bikes
Station 42: 19.00 bikes